In [1]:
import ee
import numpy as np
import matplotlib.pyplot as plt
import shapefile as sf
import pandas as pd
import datetime

In [2]:
ee.Authenticate()

True

In [ ]:
# Rachele's project
ee.Initialize(project = 'sediment-473319')

In [ ]:
# David's project
ee.Initialize(project = 'ee-davidmkahler-limpopo')

In [4]:
aoi=ee.Geometry.Rectangle([31.71630970,-24.05487960,31.72086902,-24.05871824])

In [ ]:
# Actual analysis
# Sensors when in the rivers in 2021, start there and run to present.
start = pd.date_range(start= '2025-01-01' , end='2025-12-27' , 
              freq='5d')
end = pd.date_range(start='2025-01-05' , end='2025-12-31' , 
              freq='5d')
#print(len(start))
#print(len(end))
dates = pd.DataFrame ({'start': start ,  'end': end})
print(dates)

        start        end
0  2025-01-01 2025-01-05
1  2025-01-06 2025-01-10
2  2025-01-11 2025-01-15
3  2025-01-16 2025-01-20
4  2025-01-21 2025-01-25
..        ...        ...
68 2025-12-07 2025-12-11
69 2025-12-12 2025-12-16
70 2025-12-17 2025-12-21
71 2025-12-22 2025-12-26
72 2025-12-27 2025-12-31

[73 rows x 2 columns]


In [5]:
# Test dates
start = pd.date_range(start= '2017-08-13' , end='2017-08-13' , 
              freq='5d')
end = pd.date_range(start='2017-08-18' , end='2017-08-18' , 
              freq='5d')
dates = pd.DataFrame ({'start': start ,  'end': end})
print(dates)

       start        end
0 2017-08-13 2017-08-18


In [75]:
# Define function
def testGEE(startDate, endDate):
    # Define source data
    image = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')\
        .filterDate(startDate, endDate)\
        .select('B1','B2','B3', 'B4', 'B5','B6','B7','B8','B9','B11','B12')\
        .filterBounds(aoi)
    mosaic = image.median().reproject(crs='EPSG:32736', scale=10) # This allows us to set the resolution.
    band_arrs = mosaic.sampleRectangle(region=aoi, defaultValue = 0)
    # Get individual band arrays.
    band_arr_b2 = band_arrs.get('B2')   # Blue
    band_arr_b3 = band_arrs.get('B3')   # Green
    band_arr_b4 = band_arrs.get('B4')   # Red
    # Transfer the arrays from server to client and cast as np array.
    b2 = np.array(band_arr_b2.getInfo())   # b2  Blue
    b3 = np.array(band_arr_b3.getInfo())   # b3  Green
    b4 = np.array(band_arr_b4.getInfo())   # b4  Red
    # Export RGB
    np_arr_b4 = np.expand_dims(b4, 2)
    np_arr_b3 = np.expand_dims(b3, 2)
    np_arr_b2 = np.expand_dims(b2, 2)
    rgb_img = np.concatenate((np_arr_b4, np_arr_b3, np_arr_b2), 2)
    rgb_img = (255*((rgb_img)/3000)).astype('uint8')
    plt.imshow(rgb_img)
    plt.show()

In [ ]:
def pulldata(startDate, endDate):
    "COPERNICUS/S2_SR_HARMONIZED"
    # Defining image collection to draw from
    image = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')\
        .filterDate(startDate, endDate)\
        .select('B1','B2','B3', 'B4', 'B5','B6','B7','B8','B9','B11','B12')\
        .filterBounds(aoi)\
        .filterMetadata('CLOUDY_PIXEL_PERCENTAGE', 'less_than', 10)
    med = image.median().reproject(crs='EPSG:32736', scale=10) # This allows us to set the resolution.
    #mosaic = med.sampleRectangle(region=aoi, defaultValue = 0)
    clipped_image = med.clip(aoi)#.select(['B1','B2','B3','B4','B5','B6','B7','B8','B8A','B9','B11','B12'])

    # Exporting image
    task = ee.batch.Export.image.toDrive(image=clipped_image,
                                     description=f'River_Images',
                                     scale=15,
                                     region=aoi,
                                     fileNamePrefix=f'Balule_{startDate}',
                                     crs='EPSG:4326',
                                     fileFormat='GeoTIFF',
                                     folder='Turbidity_Images')
    task.start()

In [83]:
def getDates(startDate, endDate):
    "COPERNICUS/S2_SR_HARMONIZED"
    # Defining image collection to draw from
    image = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')\
        .filterDate(startDate, endDate)\
        .filterBounds(aoi)\
        .first()

    # Determine date of image
    acquire = image.get('system:time_start')#.getInfo()
    acquire_date = ee.Date(acquire).format('YYYY-MM-dd HH:mm:ss')
    print(f"Acquisition date: {acquire_date.getInfo()}")

In [ ]:
def getDates(startDate, endDate):
    image = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')\
        .filterDate(startDate, endDate)\
        .filterBounds(aoi)\
        .first()
    # 2. Get the Metadata Property and Format the Date
    if image is not None:
    # Use .getInfo() to fetch the metadata property value from the EE server
    # The property system:time_start returns the time in milliseconds since epoch
        time_start_ms = image.get('system:time_start').getInfo()

    # Cast the epoch time to an ee.Date object for formatting (optional, but good practice)
    # Note: ee.Date() returns an Earth Engine object, not a Python object
        ee_date = ee.Date(time_start_ms)
    
    # 3. Print the Formatted Date (using the ee.Date object)
    # The .format() method requires a call to .getInfo() to execute and return a Python string
        #formatted_date_ee = ee_date.format('YYYY-MM-dd HH:mm:ss').getInfo()
        #print(f"Sentinel-2 Image Acquisition Date (via ee.Date): {formatted_date_ee}")

    # --- Alternative: Use standard Python datetime for conversion ---
    # Convert milliseconds to seconds (standard Python datetime uses seconds)
        time_start_s = time_start_ms / 1000 
    
    # Convert the Unix timestamp (seconds) to a standard Python datetime object
        python_date = datetime.datetime.fromtimestamp(time_start_s)
    
    # Format the Python datetime object
        formatted_date_py = python_date.strftime('%Y-%m-%d %H:%M:%S')
        print(f"For file: {startDate}, Sentinel-2 Acquisition: {formatted_date_py}")
    
    else:
        print("For file: {startDate}, No image/date found.")

# Example Output:
# Sentinel-2 Image Acquisition Date (via ee.Date): 2023-08-15 18:59:08
# Sentinel-2 Image Acquisition Date (via Python datetime): 2023-08-15 18:59:08

In [7]:
# Run over all dates
for i in range(len(dates)):
    # testGEE(dates["start"][i], dates["end"][i])
    # pulldata(dates["start"][i], dates["end"][i])
    getDates(dates["start"][i], dates["end"][i])

Sentinel-2 Image Acquisition Date (via Python datetime): 2017-08-17 03:06:47


In [ ]:
#Olifants River at Balule 
#7377-609302